# Loss Network with Finite Capacity Region (NC Solver)

This example demonstrates the NC solver's ability to analyze open loss networks using the Erlang fixed-point approximation. The model has a multiclass Delay node inside an FCR with DROP policy.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.nc.*
import jline.solvers.jmt.*
import kotlin.math.abs

In [ ]:
// Create network
val model = Network("FCR Loss Network")

In [ ]:
// Add nodes
val source = Source(model, "Source")
val delay = Delay(model, "Delay")
val sink = Sink(model, "Sink")

In [ ]:
// Add job classes
val class1 = OpenClass(model, "Class1", 0)
val class2 = OpenClass(model, "Class2", 1)

In [ ]:
// Set arrival and service rates
val lambda1 = 0.3
val lambda2 = 0.2
val mu1 = 1.0
val mu2 = 0.8

source.setArrival(class1, Exp(lambda1))
source.setArrival(class2, Exp(lambda2))
delay.setService(class1, Exp(mu1))
delay.setService(class2, Exp(mu2))

In [ ]:
// Create routing matrix
val P = model.initRoutingMatrix()
P.set(class1, class1, source, delay, 1.0)
P.set(class1, class1, delay, sink, 1.0)
P.set(class2, class2, source, delay, 1.0)
P.set(class2, class2, delay, sink, 1.0)
model.link(P)

In [ ]:
// Add finite capacity region with constraints
// When region is full, arriving jobs are dropped (lost)
val fcr = model.addRegion(listOf(delay))
fcr.setGlobalMaxJobs(5)            // Global: max 5 jobs in region
fcr.setClassMaxJobs(class1, 3)     // Class1: max 3 jobs
fcr.setClassMaxJobs(class2, 3)     // Class2: max 3 jobs
fcr.setDropRule(class1, true)      // true = drop jobs
fcr.setDropRule(class2, true)      // true = drop jobs

In [ ]:
// Run NC solver (uses Erlang fixed-point for loss networks)
println("Running NC solver (lossn method)...")
val solverNC = NC(model)
val avgTableNC = solverNC.avgTable

println("\nNC Results:")
avgTableNC.print()

In [ ]:
// Run JMT for comparison
println("Running JMT for comparison...")
val solverJMT = JMT(model, "seed", 23000, "samples", 500000)
val avgTableJMT = solverJMT.avgTable

println("\nJMT Results:")
avgTableJMT.print()

In [ ]:
// Compare results
println("\n=== Comparison ===")
println(String.format("%-15s %12s %12s %12s", "Metric", "NC", "JMT", "Rel Err"))
println("-".repeat(55))

// Extract Delay node throughputs
val delayRowsNC = avgTableNC.filter { it["Station"] == "Delay" }
val delayRowsJMT = avgTableJMT.filter { it["Station"] == "Delay" }

for (i in 0 until delayRowsNC.size) {
    val tputNC = delayRowsNC[i]["Tput"] as Double
    val tputJMT = delayRowsJMT[i]["Tput"] as Double
    val relErr = abs(tputNC - tputJMT) / tputJMT * 100
    val className = delayRowsNC[i]["JobClass"] as String
    println(String.format("Tput %-10s %12.4f %12.4f %10.2f%%", className, tputNC, tputJMT, relErr))
}

println("\nNote: NC uses analytical Erlang fixed-point approximation.")
println("JMT uses discrete-event simulation.")

## Explanation

This example demonstrates a loss network analyzed with the NC (Normalizing Constant) solver:

1. **Loss Network**: An FCR with DROP policy creates a loss network where jobs are lost when capacity is reached.

2. **NC Solver**: Uses Erlang fixed-point approximation to analytically compute performance metrics for loss networks.

3. **Comparison**: Results are compared with JMT (simulation) to validate the analytical approximation accuracy.

The model has two job classes arriving at a Delay node with finite capacity. When the region is full, arriving jobs are dropped (lost) rather than waiting.